In [596]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [597]:
data, target = load_breast_cancer()["data"], load_breast_cancer()["target"]
X_train, X_test, y_train, y_test = train_test_split(data, target)

In [598]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

def sigmoid(x):
    return np.where(x < 0, np.exp(x)/(1 + np.exp(x)), 1/(1 + np.exp(-x)))

class MLP(BaseEstimator, TransformerMixin):
    
    def __init__(self, layer_sizes, layer_activations = [], learning_rate = 1.0):
        self.layer_sizes = layer_sizes
        self.layer_activations = layer_activations
        self.learning_rate = 1.0
    
    def predict(self, X):
        result = self.feed_forward(self.scaler.transform(X)).reshape(-1,1)
        result[result > 0.5] = 1
        result[result != 1]  = 0
        return result
        
    
    def feed_forward(self, X, return_intermediate = False):
        
        if return_intermediate:
            intermediate = []
        
        feed = X
        
        for w, act in zip(self.weights, self.layer_activations):
            bias_column = np.ones((X.shape[0], 1))
            feed = np.append(bias_column, feed, axis=1)
            if return_intermediate:
                intermediate.append(feed)
            feed = feed @ w
            
            if act == "sigmoid":
                feed = sigmoid(feed)
        
        if not return_intermediate:
            return feed
        return feed, intermediate
            
    
    def fit(self, X, y, n_steps = 10, warm_start = False, verbose = False):
        self.scaler = StandardScaler()
        self.scaler.fit(X)
        X = self.scaler.transform(X)
        
        if not warm_start:
            self.input_shape = X.shape[1]
            self.weights = [np.random.rand(self.input_shape+1, self.layer_sizes[0])-0.5]
        
            for l_index in range(len(self.layer_sizes)-1):
                prev_weights = np.random.rand(self.layer_sizes[l_index]+1, self.layer_sizes[l_index+1])-0.5
                self.weights.append(prev_weights)
            
        while n_steps > 0:
            if verbose:
                print("Steps remaining:", n_steps)
            result, feeds = self.feed_forward(X, True)
            yres = y.reshape(-1, 1)
            deltas = [result-yres]
            if verbose:
                print("Output cross entropy:", -1/X.shape[0]*np.sum(
                    np.multiply(yres, np.log(result)) + np.multiply(1-yres, np.log(1-result))))

            for widx in range(len(self.weights)-1, -1, -1):
                if len(deltas) > 1:
                    current_used = deltas[0][:,:-1]
                else:
                    current_used = deltas[0]
                g_prime = np.multiply(feeds[widx], 1-feeds[widx])
                gradient = current_used @ self.weights[widx].T
                new_delta = np.multiply(gradient, g_prime)
                deltas.insert(0, new_delta)
                #print(g_prime)
            
            n_steps -= 1
            
            for idx, (feed, delta) in enumerate(zip(feeds, deltas[1:])):
                #print(feed.T.shape, delta.shape, self.weights[idx].shape)
                if idx != len(deltas)-2:
                    grad = feed.T @ delta[:, :-1]
                else:
                    grad = feed.T @ delta
                self.weights[idx] -= grad*self.learning_rate/X.shape[0]
                #print(delta)
        

In [599]:
mlp = MLP([300, 300, 100, 1], ["sigmoid", "sigmoid", "sigmoid", "sigmoid"], learning_rate=0.0005)

In [601]:
mlp.fit(X_train, y_train, n_steps = 100, verbose=False)

In [602]:
result = mlp.predict(X_test)

In [603]:
from sklearn.metrics import accuracy_score

In [604]:
accuracy_score(result, y_test)

0.965034965034965

In [605]:
from tensorflow import keras

In [623]:
keras_model = keras.Sequential([
    keras.layers.Dense(300, activation="sigmoid", input_shape=X_train.shape[1:]),
    keras.layers.Dense(300, activation="sigmoid"),
    keras.layers.Dense(100, activation="sigmoid"),
    keras.layers.Dense(1, activation="sigmoid")
])

In [624]:
keras_model.compile(
    optimizer = "sgd",
    metrics = "accuracy",
    loss ="binary_crossentropy"
)
keras_model.build(input_shape = X_train.shape[1:])

In [625]:
keras_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 300)               9300      
_________________________________________________________________
dense_14 (Dense)             (None, 300)               90300     
_________________________________________________________________
dense_15 (Dense)             (None, 100)               30100     
_________________________________________________________________
dense_16 (Dense)             (None, 1)                 101       
Total params: 129,801
Trainable params: 129,801
Non-trainable params: 0
_________________________________________________________________


In [637]:
keras_model.fit(X_train, y_train, epochs=100, batch_size=None)

Epoch 1/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6588 - accuracy: 0.6150
Epoch 2/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6250 - accuracy: 0.6150
Epoch 3/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6375 - accuracy: 0.6362
Epoch 4/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6153 - accuracy: 0.6878
Epoch 5/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5894 - accuracy: 0.7019
Epoch 6/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5900 - accuracy: 0.6690
Epoch 7/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5825 - accuracy: 0.7864
Epoch 8/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5666 - accuracy: 0.7606
Epoch 9/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5530 - accuracy: 0.7559
Epoch 10/100
14/14 [==============================] - 0s 2ms/step - loss: 0.5690 - accuracy: 0.6925
Epoch 11/

14/14 [==============================] - 0s 2ms/step - loss: 0.6787 - accuracy: 0.6150
Epoch 84/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6792 - accuracy: 0.5845
Epoch 85/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6780 - accuracy: 0.6174
Epoch 86/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6717 - accuracy: 0.6150
Epoch 87/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6770 - accuracy: 0.6150
Epoch 88/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6823 - accuracy: 0.5728
Epoch 89/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6801 - accuracy: 0.6174
Epoch 90/100
14/14 [==============================] - 0s 3ms/step - loss: 0.6771 - accuracy: 0.6174
Epoch 91/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6717 - accuracy: 0.6150
Epoch 92/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6751 - accuracy: 0.5939
Epoch 93/100


In [641]:
keras_pred = (keras_model.predict(X_test) > 0.5).astype("int32")

In [643]:
accuracy_score(keras_pred, y_test)

0.9230769230769231